## Switches and flags

In [1]:
ExtractIndividualStudies = True
ExtractGroupedStudies = True
AggregateStorageLevels = True
NumSrVars = 66

## Import standard libraries

In [2]:
import os
import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')
import datetime as dt
import pandas as pd
import numpy as np
import cqwlutils as cu

## Import custom modules

In [3]:
# Import custom modules - NEED WINDOWS OS (NOTE: I had to run this twice, must check why this happens!)
from coeqwalpackage.DataExtraction import *

## Define contol file name

In [4]:
CtrlFile = 'CalSim3GroundWaterDataExtractionInitFile_v1.xlsx'
CtrlTab = 'Init'

## Read from control file

In [5]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

In [6]:
print([ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax])


['coeqwal_GW_scenario_listing_v1.xlsx', 'scenario_list', '../../CalSim3_Model_Runs/Scenarios\\coeqwal_GW_scenario_listing_v1.xlsx', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\DVDssNamesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\SVDssNamesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\IndicesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\DirNamesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\ground_water_variables_v1.xlsx', 'ground_water_variables_v1.xlsx', 'GroundWaterVars_CS3', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\VarsFrom_ground_water_variables_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\DataFrom_ground_water_variables_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\ConvertDataFrom_ground_water_variabl

## Check for output directory and create if necessary

In [7]:
# check if output directory exists
if not os.path.exists(GroupDataDirPath):
    # print warning
    print("Warning: directory " + GroupDataDirPath + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(GroupDataDirPath)


## Define Nan Values

In [8]:
# NaN values as defined by CalSim3
Nan1 = -901
Nan2 = -902

## Read indeces, dss names, directory names, start and end dates, time range

In [9]:
gw1dsshdr, gw1dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW1DssMin, GW1DssMax, hdr=True)
gw1dss_names = []
for i in range(len(gw1dssname)):
    gw1dss_names.append(gw1dssname[i][0])
gw1dss_names

['s0001_DCR2023_9.3.1_danube_hist\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0002_DCR2023_9.3.1_danube_adj\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0003_DCR2023_9.3.1_danube_cc50\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0004_DCR2023_9.3.1_danube_cc75\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0005_DCR2023_9.3.1_danube_cc95\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0006_draftDCP_NDDOff_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0007_draftDCP_NDD_SWP3000_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0008_draftDCP_NDD_SWP4500_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0009_draftDCP_NDD_SWP6000_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0010_draftDCP_NDD_SWP6000_Beth_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0011_adjBL_wTUCP\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0012_adj_SGMApump\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0013_DCR-CC75_wTU

In [10]:
gw2dsshdr, gw2dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW2DssMin, GW2DssMax, hdr=True)
gw2dss_names = []
for i in range(len(gw2dssname)):
    gw2dss_names.append(gw2dssname[i][0])
gw2dss_names

['s0001_DCR2023_9.3.1_danube_hist\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0002_DCR2023_9.3.1_danube_adj\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0003_DCR2023_9.3.1_danube_cc50\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0004_DCR2023_9.3.1_danube_cc75\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0005_DCR2023_9.3.1_danube_cc95\\Model_Files\\DSS\\input\\output\\CVGroundwaterBudget.dss',
 's0006_draftDCP_NDDOff_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0007_draftDCP_NDD_SWP3000_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0008_draftDCP_NDD_SWP4500_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0009_draftDCP_NDD_SWP6000_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0010_draftDCP_NDD_SWP6000_Beth_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0011_adjBL_wTUCP\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0012_adj_SGMApump\\Model_Files\\DSS\\output\\CVG

In [11]:
indexhdr, index_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, IndexMin, IndexMax, hdr=True)
index_names = []
for i in range(len(index_name)):
    index_names.append(index_name[i][0])
index_names

['s0001',
 's0002',
 's0003',
 's0004',
 's0005',
 's0006',
 's0007',
 's0008',
 's0009',
 's0010',
 's0011',
 's0012',
 's0013',
 's0014',
 's0015',
 's0016',
 's0018']

In [12]:
studyhdr, study_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, NameMin, NameMax, hdr=True)
study_names = []
for i in range(len(study_name)):
    study_names.append(study_name[i][0])
study_names

['9.3.1_danube_hist',
 '9.3.1_danube_adj',
 '9.3.1_danube_cc50',
 '9.3.1_danube_cc75',
 '9.3.1_danube_cc95',
 's0006_draftDCP_NDDOff_2020',
 's0007_draftDCP_NDD_SWP3000_2020',
 's0008_draftDCP_NDD_SWP4500_2020',
 's0009_draftDCP_NDD_SWP6000_2020',
 's0010_draftDCP_NDD_SWP6000_Beth_2020',
 's0011_adjBL_wTUCP',
 's0012_adj_SGMApump',
 's0013_DCR-CC75_wTUCP',
 's0014_DCR-CC75_SGMApump',
 's0015_SGMApump_wTUCP',
 's0016_DCR-CC75_SGMApump_wTUCP',
 's0018_adj_EflowsV1']

In [13]:
dirhdr, dir_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, DirMin, DirMax, hdr=True)
dir_names = []
for i in range(len(dir_name)):
    dir_names.append(dir_name[i][0])
dir_names

['s0001_DCR2023_9.3.1_danube_hist',
 's0002_DCR2023_9.3.1_danube_adj',
 's0003_DCR2023_9.3.1_danube_cc50',
 's0004_DCR2023_9.3.1_danube_cc75',
 's0005_DCR2023_9.3.1_danube_cc95',
 's0006_draftDCP_NDDOff_2020',
 's0007_draftDCP_NDD_SWP3000_2020',
 's0008_draftDCP_NDD_SWP4500_2020',
 's0009_draftDCP_NDD_SWP6000_2020',
 's0010_draftDCP_NDD_SWP6000_Beth_2020',
 's0011_adjBL_wTUCP',
 's0012_adj_SGMApump',
 's0013_DCR-CC75_wTUCP',
 's0014_DCR-CC75_SGMApump',
 's0015_SGMApump_wTUCP',
 's0016_DCR-CC75_SGMApump_wTUCP',
 's0018_adj_EflowsV1']

In [14]:
starthdr, start_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, StartMin, StartMax, hdr=True)
start_dates = []
for i in range(len(start_date)):
    start_dates.append(start_date[i][0])
datetime_start_dates = pd.to_datetime(start_dates)
# turns out that dss reading library wands a dt datetime, not pd datetime
dt_datetime_start_dates = [dt.to_pydatetime() for dt in datetime_start_dates]


In [15]:
endhdr, end_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, EndMin, EndMax, hdr=True)
end_dates = []
for i in range(len(end_date)):
    end_dates.append(end_date[i][0])
# turns out that dss reading library wands a dt datetime, not pd datetime
datetime_end_dates = pd.to_datetime(end_dates)
dt_datetime_end_dates = [dt.to_pydatetime() for dt in datetime_end_dates]


In [16]:
min_datetime = min(dt_datetime_start_dates)
print('Min time: ')
print(min_datetime)
max_datetime = max(dt_datetime_end_dates)
print('Max time: ')
print(max_datetime)


Min time: 
1921-10-31 00:00:00
Max time: 
2021-09-30 00:00:00


## Set path and file names for indeces, dss names and directory names and write files

In [17]:
gw1dss_df = (pd.DataFrame(gw1dss_names))
gw1dss_df.to_csv(GW1DssNamesOutPath)
print(GW1DssNamesOutPath)
index_df = (pd.DataFrame(gw1dss_names))
index_df.to_csv(ScenarioIndicesOutPath)
print(ScenarioIndicesOutPath)
dir_df = (pd.DataFrame(gw1dss_names))
dir_df.to_csv(DssDirsOutPath)
print(DssDirsOutPath)


../../CalSim3_Model_Runs/Scenarios\Group_Data_Extraction\DVDssNamesFrom_coeqwal_GW_scenario_listing_v1.csv
../../CalSim3_Model_Runs/Scenarios\Group_Data_Extraction\IndicesFrom_coeqwal_GW_scenario_listing_v1.csv
../../CalSim3_Model_Runs/Scenarios\Group_Data_Extraction\DirNamesFrom_coeqwal_GW_scenario_listing_v1.csv


## Read and write variables list

In [18]:
# get vars
hdr, vars = cu.read_from_excel(VarListPath, VarListTab,VarMin,VarMax,hdr=True)
gw1var_df = pd.DataFrame(data=vars, columns=hdr)
gw1var_df

,Part B:,Part C:
0,LT:E999,PUMPING
1,SR1:L1,GW_STORAGE
2,SR2:L1,GW_STORAGE
3,SR3:L1,GW_STORAGE
4,SR4:L1,GW_STORAGE
...,...,...
194,SR62:L3,GW_STORAGE
195,SR63:L3,GW_STORAGE
196,SR64:L3,GW_STORAGE
197,SR65:L3,GW_STORAGE


In [19]:
# write vars
gw1var_df.to_csv(VarOutPath)
VarOutPath

'../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\VarsFrom_ground_water_variables_v1.csv'

## Loop on DSS files, extract variables and write to scenario-specific CSVs

In [20]:
if ExtractIndividualStudies:
    for i in range(len(gw1dss_names)):
        gw1dss_name = os.path.join(ScenarioDir, gw1dss_names[i])
        datetime_start_date = dt_datetime_start_dates[i]
        datetime_end_date = dt_datetime_end_dates[i]
        print('\nReading ' + gw1dss_name + '\n')   
        df = preprocess_GW_data_study_dss(gw1var_df, gw1dss_name, datetime_start_date, datetime_end_date, addSRlevels = AggregateStorageLevels, num_vars = NumSrVars)
        file_path = gw1dss_name.replace(ModelSubPath, ExtractionSubPath).replace(".dss", ".csv")
        dir_path = os.path.dirname(file_path)
        os.makedirs(dir_path, exist_ok=True)
        print(df.tail(5))
        print('\nWriting ' + file_path + '\n')
        df.to_csv(file_path, na_rep="NaN", header=True, index=True)


Reading ../../CalSim3_Model_Runs/Scenarios\s0001_DCR2023_9.3.1_danube_hist\Model_Files\DSS\output\CVGroundwaterOut.dss

Start: 
1921-10-31 00:00:00
End: 
2021-09-30 00:00:00
Opened file: ../../CalSim3_Model_Runs/Scenarios\s0001_DCR2023_9.3.1_danube_hist\Model_Files\DSS\output\CVGroundwaterOut.dss
Valid catalog found: True 
Number of records: 124707
Took 0.11665201187133789 sec to convert paths to dataframe
Took 0.017986774444580078 sec to do group-by operation
Took 0.005950927734375 sec to add column


C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:404: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['search_col'] = ['/'.join([str(c1) for c1 in c]) if type(c)==tuple else c for c in condcat.index]
C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['full_path'] = ['/'+'/'.join([str(c[1][k]) if k !='' else '' for k in keep_parts])+'/' for c in condcat.iterrows()]


Took 0.7370789051055908 sec to rebuild modified path
Filtering path list
A                             IWFM                               \5% ...Retrieving 199 DV Time Series
B                          LT:E999                      SR10:L1   
C                          PUMPING                   GW_STORAGE   
D                             1MON                         1MON   
E          PUMPING_AT_ALL_ELEMENTS GW_STORAGE_AT_CALSIM_REGIONS   
F                          PER-CUM                      PER-CUM   
Units                       AC.FT.                       AC.FT.   
2021-05-31            -2909.803223                    4515926.0   
2021-06-30            -2372.584229                    4507483.0   
2021-07-31            -1912.345337                    4501759.5   
2021-08-31            -2008.060059                    4492035.5   
2021-09-30            -1959.298828                    4488501.0   

A                                                                     \
B              

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:404: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['search_col'] = ['/'.join([str(c1) for c1 in c]) if type(c)==tuple else c for c in condcat.index]
C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['full_path'] = ['/'+'/'.join([str(c[1][k]) if k !='' else '' for k in keep_parts])+'/' for c in condcat.iterrows()]


Took 0.7460048198699951 sec to rebuild modified path
Filtering path list
A                             IWFM                               \5% ...Retrieving 199 DV Time Series
B                          LT:E999                      SR10:L1   
C                          PUMPING                   GW_STORAGE   
D                             1MON                         1MON   
E          PUMPING_AT_ALL_ELEMENTS GW_STORAGE_AT_CALSIM_REGIONS   
F                          PER-CUM                      PER-CUM   
Units                       AC.FT.                       AC.FT.   
2021-05-31            -2521.256104                    4515646.5   
2021-06-30            -2504.480713                    4506840.0   
2021-07-31            -2001.298218                    4500518.0   
2021-08-31            -2021.137939                    4491433.5   
2021-09-30            -1919.871704                    4488017.0   

A                                                                     \
B              

...

Took 0.7629590034484863 sec to rebuild modified path
Filtering path list
A                             IWFM                               \5% ...Retrieving 199 DV Time Series
B                          LT:E999                      SR10:L1   
C                          PUMPING                   GW_STORAGE   
D                             1MON                         1MON   
E          PUMPING_AT_ALL_ELEMENTS GW_STORAGE_AT_CALSIM_REGIONS   
F                          PER-CUM                      PER-CUM   
Units                       AC.FT.                       AC.FT.   
2021-05-31            -2579.763184                    4515488.5   
2021-06-30            -2504.484131                    4506032.5   
2021-07-31            -2195.331543                    4498013.0   
2021-08-31            -3161.130859                    4490541.0   
2021-09-30            -1932.872925                    4487414.5   

A                                                                     \
B              

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:404: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['search_col'] = ['/'.join([str(c1) for c1 in c]) if type(c)==tuple else c for c in condcat.index]
C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['full_path'] = ['/'+'/'.join([str(c[1][k]) if k !='' else '' for k in keep_parts])+'/' for c in condcat.iterrows()]


Took 0.8317756652832031 sec to rebuild modified path
Filtering path list
A                             IWFM                               \5% ...Retrieving 199 DV Time Series
B                          LT:E999                      SR10:L1   
C                          PUMPING                   GW_STORAGE   
D                             1MON                         1MON   
E          PUMPING_AT_ALL_ELEMENTS GW_STORAGE_AT_CALSIM_REGIONS   
F                          PER-CUM                      PER-CUM   
Units                       AC.FT.                       AC.FT.   
2021-05-31            -2909.055664                    4491482.0   
2021-06-30            -2152.227295                    4488174.0   
2021-07-31            -1959.551025                    4475323.0   
2021-08-31            -2588.257812                    4473610.5   
2021-09-30            -1700.707642                    4465206.5   

A                                                                     \
B              

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:404: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['search_col'] = ['/'.join([str(c1) for c1 in c]) if type(c)==tuple else c for c in condcat.index]
C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\AuxFunctions.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['full_path'] = ['/'+'/'.join([str(c[1][k]) if k !='' else '' for k in keep_parts])+'/' for c in condcat.iterrows()]


Took 0.8377199172973633 sec to rebuild modified path
Filtering path list
A                             IWFM                               \5% ...Retrieving 199 DV Time Series
B                          LT:E999                      SR10:L1   
C                          PUMPING                   GW_STORAGE   
D                             1MON                         1MON   
E          PUMPING_AT_ALL_ELEMENTS GW_STORAGE_AT_CALSIM_REGIONS   
F                          PER-CUM                      PER-CUM   
Units                       AC.FT.                       AC.FT.   
2021-05-31            -2756.627197                    4491275.5   
2021-06-30            -2160.208008                    4471238.5   
2021-07-31            -2096.969238                    4448894.5   
2021-08-31            -3074.354980                    4431920.5   
2021-09-30            -1968.845581                    4425384.0   

A                                                                     \
B              

## Read all DSS files, combine variables and write to single CSV
#### Note: warnings will appear whenever a study length is shorter than the max time range

In [21]:
if ExtractGroupedStudies:
    df = preprocess_compound_GW_data_dss(gw1var_df, ScenarioDir, gw1dss_names, index_names, min_datetime, max_datetime, addSRlevels = AggregateStorageLevels, num_vars = NumSrVars)
    df.replace([Nan1, Nan2], np.nan, inplace=True)
    df.tail(5)

## Write the compund dataframe to CSV

In [22]:
# write the dataframe to CSV
if ExtractGroupedStudies:
    print('Writing ' + DataOutPath)
    df.to_csv(DataOutPath, na_rep="NaN", header=True, index=True)

In [23]:
print('Done!')

Done!
